# 資料爬蟲練習：永慶實價登錄3.0網頁內湖區地區查詢

## 1. 腳本初始化

In [13]:
source(paste0(getwd(),"/NeedPackage.R"))
source(paste0(getwd(),"/UrlXpath.R"))
source(paste0(getwd(),"/FunLibary.R"))

👍️初始化完成!! 



## 2.建構爬蟲程式

In [14]:
# 先將需要的輔助項目建立起來
first_time = Sys.time()# 用來計算整支程式的運行時間
x <- data.frame()# Start DataFrame

## 3.開始爬取資料

In [15]:
# 讀取網頁URL
start_time <- Sys.time()# 用來計算讀取網頁的運行時間
for(i in 1:8){# 搜尋結果一共有8頁
  if(i == 1){
    paths <- url_fun("")
  }else{
    paths <- url_fun(i)
  }
  # 建構所需要的資料
  htmlcontent <- read_html(paths)
  Dates <- htmlcontent%>%
    html_nodes(xpath = path_Date)%>%
    html_text()# 成交
  Types <- htmlcontent%>%
    html_nodes(xpath = path_Type)%>%
    html_text()# 型態
  Locals <- htmlcontent%>%
    html_nodes(xpath = path_Local)%>%
    html_text()# 地址
  Closings <- htmlcontent%>%
    html_nodes(xpath = path_Closing)%>%
    html_text()# 成交價
  BuildPings <- htmlcontent%>%
    html_nodes(xpath = path_BuildPing)%>%
    html_text()# 建坪
  Prices <- htmlcontent%>%
    html_nodes(xpath = path_Price)%>%
    html_text()# 單價
  LocalPings <- htmlcontent%>%
    html_nodes(xpath = path_LocalPing)%>%
    html_text()# 地坪
  Floors <- htmlcontent%>%
    html_nodes(xpath = path_Floor)%>%
    html_text()# 樓別
  Olds <- htmlcontent%>%
    html_nodes(xpath = path_Old)%>%
    html_text()# 屋齡
  ParkingSpaces <- htmlcontent%>%
    html_nodes(xpath = path_ParkingSpace)%>%
    html_text()# 車位
  PathNum <- substr(paths,nchar(paths)-20,nchar(paths))
  cat("⏳️第",i,"頁讀取中...，已經過",start_time-Sys.time(),"秒\n")# 計算每次讀取時間，觀察是否有異常的執行時間
  x <- rbind(x,data.frame(Dates,Types,Locals,Closings,BuildPings,Prices,LocalPings,Floors,Olds,ParkingSpaces))# 存入資料
}# Finish DataFrame
cat("讀取完成!!!")

⏳️第 1 頁讀取中...，已經過 -1.009829 秒
⏳️第 2 頁讀取中...，已經過 -1.764697 秒
⏳️第 3 頁讀取中...，已經過 -2.88725 秒
⏳️第 4 頁讀取中...，已經過 -3.71125 秒
⏳️第 5 頁讀取中...，已經過 -4.852705 秒
⏳️第 6 頁讀取中...，已經過 -5.559453 秒
⏳️第 7 頁讀取中...，已經過 -7.105086 秒
⏳️第 8 頁讀取中...，已經過 -8.419255 秒
讀取完成!!!

## 4.數據清理

In [16]:

head(x)
# 在jupyter上無法明顯看出資料是否髒亂，此處是為了建立良好習慣。
# 實際打開excel會發現撈下來的資料並不乾淨。

,Dates,Types,Locals,Closings,BuildPings,Prices,LocalPings,Floors,Olds,ParkingSpaces
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,10810,廠辦,內湖區行愛路141巷11~20號,"7,800萬 (含車位480萬)",219.57坪 (含車位44.47坪),41.8萬,45.91坪,4~4/6,8.0年,有
2,10808,廠辦,金矽谷NO.3 內湖區陽光街345巷11~20號,"6,820萬 (含車位600萬)",201.99坪 (含車位42.30坪),39.0萬,33.63坪,4~4/7,19.0年,有
3,10808,電梯大樓,長虹凌雲 內湖區文德路61~70號 格局：4房(室)2廳2衛,"6,350萬 (含車位750萬)",125.36坪 (含車位37.11坪),63.5萬,11.66坪,7~7/17,9.0年,有
4,10808,電梯大樓,富邦大衛營領袖特區 內湖區成功路五段450巷23弄11~20號 格局：4房(室)2廳3衛,"9,088萬 (含車位900萬)",124.25坪 (含車位30.89坪),87.7萬,23.82坪,15~15/17,11.0年,有
5,10811,電梯大樓,香堤苑 內湖區行善路281巷19弄1~10號 格局：4房(室)2廳3衛,"6,650萬 (含車位500萬)",116.92坪 (含車位21.30坪),64.3萬,22.53坪,2~2/14,9.0年,有
6,10811,廠辦,內湖區文湖街60巷31~40號,"3,470萬",113.66坪,30.5萬,50.24坪,4~4/5,29.0年,無


In [17]:
# 開始清理資料
for(i in 1:ncol(x)){# 刪除\n&\r&空白鍵
  x[,i] <- Etl_nr(x,i)
}

## 5.匯出資料

In [18]:
# 將清理完的資料匯到指定的工作目錄(資料夾)
Today <- Sys.Date()
Insurance_name <- paste0(getwd(),"/Crawler_Data/(",Today,")Crawler_YoungChing_Neihu.csv")
write.table(x,Insurance_name,quote=TRUE,sep = ",",na="NaN",fileEncoding="utf-8")# Output data to csv
cat("恭喜完成!!!總共花費",first_time-Sys.time(),"秒，請至目標資料夾(Crawler_Data)查看檔案!")

恭喜完成!!!總共花費 -42.94183 秒，請至目標資料夾(Crawler_Data)查看檔案!

In [19]:
# 成果
head(x)

,Dates,Types,Locals,Closings,BuildPings,Prices,LocalPings,Floors,Olds,ParkingSpaces
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,10810,廠辦,內湖區行愛路141巷11~20號,"7,800萬(含車位480萬)",219.57坪(含車位44.47坪),41.8萬,45.91坪,4~4/6,8.0年,有
2,10808,廠辦,金矽谷NO.3內湖區陽光街345巷11~20號,"6,820萬(含車位600萬)",201.99坪(含車位42.30坪),39.0萬,33.63坪,4~4/7,19.0年,有
3,10808,電梯大樓,長虹凌雲內湖區文德路61~70號格局：4房(室)2廳2衛,"6,350萬(含車位750萬)",125.36坪(含車位37.11坪),63.5萬,11.66坪,7~7/17,9.0年,有
4,10808,電梯大樓,富邦大衛營領袖特區內湖區成功路五段450巷23弄11~20號格局：4房(室)2廳3衛,"9,088萬(含車位900萬)",124.25坪(含車位30.89坪),87.7萬,23.82坪,15~15/17,11.0年,有
5,10811,電梯大樓,香堤苑內湖區行善路281巷19弄1~10號格局：4房(室)2廳3衛,"6,650萬(含車位500萬)",116.92坪(含車位21.30坪),64.3萬,22.53坪,2~2/14,9.0年,有
6,10811,廠辦,內湖區文湖街60巷31~40號,"3,470萬",113.66坪,30.5萬,50.24坪,4~4/5,29.0年,無


![ChessUrl](https://imgur.com/QRWSove.gif "chess") 